In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
import os

def solve_vec(method, f, y0, rng, h):
    t0, tn = rng
    t = np.arange(t0,tn,h)
    y = np.zeros((len(t),)+y0.shape, np.float64)
    y[0] = y0
    
    for ti in range(1, len(t)):
        y[ti] = method(f, t[ti-1], y[ti-1], h)
    return t, y

def euler_method(f, t, y, h):
    return y + h * f(t, y)

def midpoint_method(f, t, y, h):
    K1 = f(t, y)
    K2 = f(t+h*0.5, y+(h*0.5*K1))
    return y + h * K2

def RK4_method(f, t, y, h):
    K1 = f(t, y)
    K2 = f(t+h*0.5, y+(h*0.5*K1))
    K3 = f(t+h*0.5, y+(h*0.5*K2))
    K4 = f(t+h, y+(h*K3))
    return y + h*(K1/6+K2/3+K3/3+K4/6)

In [2]:
def render_frame(y, l, scale, shape):
    theta = y[0]
    r = y[1]
    img = np.ones(shape)*[32,40,48]
    
    pos = (shape[0]//2+int(scale*(l+r)*np.sin(theta)), shape[0]//2+int(scale*(l+r)*np.cos(theta)))
    radius = 12
    cv2.line(img, (shape[0]//2,shape[0]//2), pos, [64,64,64], 2)
    cv2.circle(img, pos, radius, [82,16,213], -1)
    cv2.circle(img, (pos[0]+radius//3,pos[1]-radius//3), radius//3, [235,240,245], -1)
    return img

def balls_animation(t, y, l, size, pre=''):
    path = 'output/'+pre+'_'+str(time.time())+'.avi'
    video = cv2.VideoWriter(
        path,
        cv2.VideoWriter_fourcc(*'MJPG'),
        len(t)//t[-1], (size,size))
    for i in range(len(t)):
        frame = render_frame(y[i], l, size/4, (size,size,3))
        video.write(frame.astype(np.uint8))
    video.release()
    
    path_mp4 = path[:-4]+'.mp4'
    os.popen("ffmpeg -i '{input}' -ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4 '{path_mp4}'".format(input = path, path_mp4=path_mp4))
    os.remove(path)
    return path_mp4

## Zadanie 4. Sprężyste wahadło.

- Rozważyć model z rozciągliwą nicią.
- Zasymulować ruch wahadła sprężystego.

$ \theta'' = -\frac{g \cdot sin\theta + 2 r'\theta'}{L+r} $

$ r'' = g \cdot cos\theta +(L+r)\theta'^2 - \frac{k}{m}r $

In [3]:
g = 9.80665
L = 1
k = 50
m = 1
def f(t, y):
    theta = y[0]
    r = y[1]
    return np.array([
            [theta[1], -(g*np.sin(theta[0])+2*theta[1]*r[1])/(L+r[0])],
            [r[1], g*np.cos(theta[0]) + (L+r[0])*theta[1]**2 - (k*r[0])/m]
        ])

# Initial state
y0 = np.array([[3.14, 0], [0,0]], np.float64)

rng = (0,40)
h = 1/30.

t, y = solve_vec(RK4_method, f, y0, rng, h)
path = balls_animation(t, y[:,:,0], L, 800, pre='t4')

HTML('<video width="800" height="800" controls><source src="'+path+'" type="video/mp4"></video>')